<a href="https://colab.research.google.com/github/PARTHIBAN-007/Hands-ON-LLMs/blob/main/FineTuning_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets>=2.18.0 transformers>=4.38.2 sentence-transformers>=2.5.1 setfit>=1.0.3 accelerate>=0.27.2 seqeval>=1.2.2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

tomatoes = load_dataset("rotten_tomatoes")
train_data , test_data = tomatoes["train"] ,tomatoes["test"]

README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoTokenizer , AutoModelForSequenceClassification
model_id = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_id,num_labels= 2)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer =tokenizer)

def text_preprocess(examples):
  return tokenizer(examples["text"],truncation=True)

tokenized_train = train_data.map(text_preprocess,batched=True)
tokenized_test = test_data.map(text_preprocess,batched=True)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

In [8]:
import numpy as np
import evaluate

def compute_metrics(eval_pred):
  logits , labels = eval_pred
  predictions = np.argmax(logits,axis=-1)

  load_f1 = evaluate.load("f1")
  f1 = load_f1.compute(predictions=predictions,references=labels)["f1"]
  return {"f1":f1}